# **Unstructured RAG**
Unstructured or (Semi-Structured) RAG is a method designed to handle documents that combine text, tables, and images. It addresses challenges like broken tables caused by text splitting and the difficulty of embedding tables for semantic search.

Here we are using unstructured.io to parse and separate text, tables, and images.

Tool Reference: [Unstructured](https://unstructured.io/)

## **Initial Setup**

In [ ]:
! pip install --q athina faiss-gpu pytesseract unstructured-client "unstructured[all-docs]"

In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['ATHINA_API_KEY'] = userdata.get('ATHINA_API_KEY')

## **Indexing**

In [4]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
 # load and extract images, tables, and chunk text
from unstructured.partition.pdf import partition_pdf

filename = "/content/sample.pdf"

pdf_elements = partition_pdf(
    filename=filename,
    extract_images_in_pdf=True,
    strategy = "hi_res",
    hi_res_model_name="yolox",
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=3000,
    combine_text_under_n_chars=200,
)

In [7]:
# check unique categories
from collections import Counter
category_counts = Counter(str(type(element)) for element in pdf_elements)
unique_categories = set(category_counts)
category_counts

Counter({"<class 'unstructured.documents.elements.CompositeElement'>": 14,
         "<class 'unstructured.documents.elements.TableChunk'>": 2})

In [8]:
# extract unique types
unique_types = {el.to_dict()['type'] for el in pdf_elements}
unique_types

{'CompositeElement', 'Table'}

In [9]:
# # display images from pdf
# from IPython.display import Image, display
# image_files = os.listdir('/content/figures')
# image_files = [os.path.join('/content/figures', image_file) for image_file in image_files]

# for image_file in image_files:
#     display(Image(filename=image_file))

In [10]:
# convert pdf_elements to langchain documents
from langchain.schema import Document
documents = [Document(page_content=el.text, metadata={"source": filename}) for el in pdf_elements]

## **Vector Store**

In [11]:
# create vectorstore
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

## **Retriever**

In [12]:
# create retriever
retriever = vectorstore.as_retriever()

## **RAG Chain**

In [13]:
# load llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [14]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the provided context, which can include text and tables.
Use the provided context to answer the question.
Question: {input}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# response
response = rag_chain.invoke("Compare all the Training Results on MATH Test Set")
response

'To compare all the Training Results on the MATH Test Set, we can look at the results from Table 6 in the provided context. The results are as follows:\n\n- deepseek-sft-abel:\n   - SFT-phase1: 0.372\n   - SFT-phase2-shortcutLearning: 0.386\n   - SFT-phase2-journeyLearining: 0.470\n   - DPO: 0.472\n\n- deepseek-sft-prm800k:\n   - SFT-phase1: 0.290\n   - SFT-phase2-shortcutLearning: 0.348\n   - SFT-phase2-journeyLearining: 0.428\n   - DPO: 0.440\n\nBased on these results, we can see that Journey Learning led to significant improvements compared to Shortcut Learning on both models, with gains of +8.4 and +8.0 on deepseek-sft-abel and deepseek-sft-prm800k, respectively. The DPO results were also provided for comparison.'

## **Preparing Data for Evaluation**

In [16]:
# create dataset
question = ["Compare all the Training Results on MATH Test Set"]
response = []
contexts = []

# Inference
for query in question:
  response.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "query": question,
    "response": response,
    "context": contexts,
}

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [17]:
# create dataset
from datasets import Dataset
dataset = Dataset.from_dict(data)

In [18]:
# create dataframe
import pandas as pd
df = pd.DataFrame(dataset)

In [19]:
df

,query,response,context
0,Compare all the Training Results on MATH Test Set,To compare all the Training Results on the MAT...,[The results of our experiments are shown in T...


In [20]:
# Convert to dictionary
df_dict = df.to_dict(orient='records')

# Convert context to list
for record in df_dict:
    if not isinstance(record.get('context'), list):
        if record.get('context') is None:
            record['context'] = []
        else:
            record['context'] = [record['context']]

## **Evaluation in Athina AI**

We will use **Does Response Answer Query** eval here. It Checks if the response answer the user's query. To learn more about this. Please refer to our [documentation](https://docs.athina.ai/api-reference/evals/preset-evals/overview) for further details.

In [21]:
# set api keys for Athina evals
from athina.keys import AthinaApiKey, OpenAiApiKey
OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [22]:
# load dataset
from athina.loaders import Loader
dataset = Loader().load_dict(df_dict)

In [23]:
# evaluate
from athina.evals import DoesResponseAnswerQuery
DoesResponseAnswerQuery(model="gpt-4o").run_batch(data=dataset).to_df()

You can view your dataset at: https://app.athina.ai/develop/e5dec38c-c58c-412d-b910-588d97ccd090


,query,context,response,expected_response,display_name,failed,grade_reason,runtime,model,passed
0,Compare all the Training Results on MATH Test Set,"[The results of our experiments are shown in Table 6. All results are tested on the MATH test set, using a re-divided subset from PRM800K, which includes 500 examples. The results show that Journey Learning led to significant improvements compared to Shortcut Learning, with gains of +8.4 and +8.0 on the deepseek-sft-abel and deepseek-sft-prm800k models, respectively, demonstrating the effectiveness of our proposed Journey Learning method. However, the improvement from DPO was more modest, an...","To compare all the Training Results on the MATH Test Set, we can look at the results provided in Table 6 from the context. The results for the different models on the MATH test set are as follows:\n\n- deepseek-sft-abel: SFT-phase1 = 0.372, SFT-phase2-shortcutLearning = 0.386, SFT-phase2-journeyLearining = 0.470, DPO = 0.472\n- deepseek-sft-prm800k: SFT-phase1 = 0.290, SFT-phase2-shortcutLearning = 0.348, SFT-phase2-journeyLearining = 0.428, DPO = 0.440\n\nFrom these results, we can see that...",None,Does Response Answer Query,False,"The response provides a detailed comparison of the training results on the MATH test set for two models, deepseek-sft-abel and deepseek-sft-prm800k. It includes specific performance metrics for different phases and methods, such as SFT-phase1, SFT-phase2-shortcutLearning, SFT-phase2-journeyLearning, and DPO. Additionally, it highlights the improvements observed with the Journey Learning method compared to Shortcut Learning, which directly addresses the user's query about comparing training r...",1910,gpt-4o,1.0
